In [1]:
import scipy
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
rows=64
cols=64
channels=3

train_dir='train_data/'
test_dir='test_data/'

train_images=[train_dir+i for i in os.listdir(train_dir)]
test_images=[test_dir+i for i in os.listdir(test_dir)]

In [3]:
def read_image(image):
    img=cv2.imread(image,1)
    return cv2.resize(img,(rows,cols),interpolation=cv2.INTER_CUBIC)

In [4]:
def prepare_data(images):
    m=len(images)
    X=np.zeros((m,rows,cols,channels),dtype=np.uint8)
    y=np.zeros((1,m))
    for i,image_file in enumerate(images):
        X[i,:]=read_image(image_file)
        if 'cat' in image_file.lower():
            y[0,i]=1
        elif 'dog' in image_file.lower():
            y[0,i]=0
    return X,y
            

In [5]:
train_set,train_labels=prepare_data(train_images)
test_set,test_labels=prepare_data(test_images)

train_set_flatten=train_set.reshape(train_set.shape[0],channels*rows*cols).T
test_set_flatten=test_set.reshape(test_set.shape[0],-1).T

train_set=train_set_flatten/255
test_set=test_set_flatten/255

In [6]:
def sigmoid(z):
    res=1/(1+np.exp(-z))
    return res

In [7]:
def propagation(w,b,X,Y):
    m=X.shape[1]
    
    Z=np.dot(w.T,X)+b
    A=sigmoid(Z)
    cost=(-np.sum(Y*np.log(A)+(1-Y)*np.log(1-A)))/m
    
    dw=(np.dot(X,(A-Y).T))/m
    db=np.average(A-Y)
    
    cost=np.squeeze(cost)
    
    grads={'dw':dw,'db':db}
    
    return grads,cost
    

In [8]:
def optimizer(w,b,X,Y,num_iteration,learning_rate,print_cost=False):
    costs=[]
    for i in range(num_iteration):
        grads,cost=propagation(w,b,X,Y)
        
        dw=grads['dw']
        db=grads['db']
        
        w=w-learning_rate*dw
        b=b-learning_rate*db
        
        if i%100 ==0:
            costs.append(cost)
            
    params={'w':w,'b':b}
        
    grads={'dw':dw,'db':db}
        
    return params,grads,costs

In [9]:
def predict(w,b,X):
    m=X.shape[1]
    y_predict=np.zeros((1,m))
    w=w.reshape(X.shape[0],1)
    
    z=np.dot(w.T,X)+b
    A=sigmoid(z)
    
    for i in range(A.shape[1]):
        if A[0,i]>0.5:
            y_predict[[0],[i]]=1
        else:
            y_predict[[0],[i]]=0
    return y_predict
    

In [10]:
def initialize(dim):
    w=np.zeros((dim,1))
    b=0
    return w,b

In [11]:
def model(X_train,Y_train,X_test,Y_test,num_iteration=200,learning_rate=0.5,print_cost=False):
    w,b=initialize(X_train.shape[0])
    params,grads,cost=optimizer(w,b,X_train,Y_train,num_iteration,learning_rate,print_cost)
    w=params['w']
    b=params['b']
    
    Y_predict_train=predict(w,b,X_train)
    Y_predict_test=predict(w,b,X_test)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_predict_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_predict_test - Y_test)) * 100))
    
    res_dict = {"cost": cost,
            "Y_predict_test": Y_predict_test,
            "Y_predict_train": Y_predict_train,
            "w": w,
            "b": b,
            "learning_rate": learning_rate,
            "num_iterations:": num_iteration}
    return res_dict

In [12]:
d=model(train_set,train_labels,test_set,test_labels,num_iteration=3000,learning_rate=0.003,print_cost=True)


train accuracy: 67.075 %
test accuracy: 60.05 %


In [13]:
test_img = "cat.jpg"
test_img = read_image(test_img).reshape(1, rows*cols*channels).T
prediction = predict(d["w"], d["b"], test_img)
print(np.squeeze(prediction))

0.0
